In [67]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import re
import time
import pandas as pd

In [68]:
service = Service('/users/paulj/chromedriver')
driver = webdriver.Chrome(service=service)
driver.get("https://store.steampowered.com/search/?sort_by=Released_DESC&filter=topsellers&supportedlang=english")

element = driver.find_element(By.ID, 'search_resultsRows')
row_dict = {}
outside_array = []

# scroll to bottom
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    # Scroll down to the bottom.
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(3)

    # Calculate new scroll height and compare with last scroll height.
    new_height = driver.execute_script("return document.body.scrollHeight")

    if new_height == last_height:

        break

    last_height = new_height

for row in element.find_elements(By.CSS_SELECTOR, 'a'):
    # row.find_element(By.CLASS_NAME, 'title').get_attribute('innerHTML')
    # row.get_attribute(By.ID, 'data-ds-tagids')
    row_dict[row.find_element(By.CLASS_NAME, 'title').get_attribute('innerHTML')] = row.get_attribute('data-ds-tagids')
    inside_array = []
    inside_array.append(row.find_element(By.CLASS_NAME, 'title').get_attribute('innerHTML'))
    inside_array.append(row.get_attribute('data-ds-tagids'))
    # inside_array.append(row.find_element(By.CLASS_NAME, ''))
    if len(row.find_elements(By.CLASS_NAME, 'win')) > 0:
        inside_array.append("windows")
    else:
        inside_array.append("")
    if len(row.find_elements(By.CLASS_NAME, 'mac')) > 0:
        inside_array.append("mac")
    else:
        inside_array.append("")
    if len(row.find_elements(By.CLASS_NAME, 'linux')) > 0:
        inside_array.append("linux")
    else:
        inside_array.append("")
    
    if len(row.find_elements(By.CLASS_NAME, 'positive')) > 0 or len(row.find_elements(By.CLASS_NAME, 'mixed')) > 0 or len(row.find_elements(By.CLASS_NAME, 'negative')) > 0:
        review = row.find_element(By.CLASS_NAME, 'search_review_summary').get_attribute('data-tooltip-html')
        numbers = re.findall(r'\d+', review)
        inside_array.append(review.split('<br>')[0])
        inside_array.append(numbers[0] + '%' + ' of positive reviews')
        inside_array.append(numbers[1] + " total reviews")
    else:
        inside_array.append("Not Rated")
        inside_array.append("Not Rated")
        inside_array.append("Not Rated")

    inside_array.append(row.find_element(By.CLASS_NAME, 'search_released').get_attribute('innerHTML'))
    if len(row.find_element(By.CLASS_NAME, 'search_discount').get_attribute('innerHTML').strip()) > 0: 
        inside_array.append('-' + row.find_element(By.CLASS_NAME, 'search_discount').get_attribute('innerHTML').split('%')[-2][-2:] + '%')
    else:
        inside_array.append("No Discount")
    inside_array.append(row.find_element(By.CLASS_NAME, 'search_price').get_attribute('innerHTML').split('$')[-1].strip())
    outside_array.append(inside_array)


In [69]:
outside_array[26]

['Destiny 2: Lightfall',
 '[19,21,113,1663,1695,353880,1754]',
 'windows',
 '',
 '',
 'Positive',
 '95% of positive reviews',
 '22 total reviews',
 'Feb 28, 2023',
 'No Discount',
 '49.99']

In [5]:
# scroll to bottom
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    # Scroll down to the bottom.
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(3)

    # Calculate new scroll height and compare with last scroll height.
    new_height = driver.execute_script("return document.body.scrollHeight")

    if new_height == last_height:

        break

    last_height = new_height

3271

480042